<a href="https://colab.research.google.com/github/VerlonRoelMBINGUI/Computer_Vision_Project_AMMI2023/blob/main/Bourbaki_CV_Cassava_Project_Kaggle_Good_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Team members :

1. Dieu - Donne FANGNON
2. Verlon Roel MBINGUI

##Cassava Disease Classification
Classify pictures of cassava leaves into 1 of 4 disease categories (or healthy)


AMMI 2023 Competition : rank 1/20

In [ ]:
!pip install ttach

In [ ]:
!pip install git+https://github.com/gbaydin/hypergradient-descent.git

from hypergrad import SGDHD, AdamHD

  Cloning https://github.com/gbaydin/hypergradient-descent.git to /tmp/pip-req-build-g62ecxu7
  Running command git clone --filter=blob:none --quiet https://github.com/gbaydin/hypergradient-descent.git /tmp/pip-req-build-g62ecxu7
  Resolved https://github.com/gbaydin/hypergradient-descent.git to commit 020d6080c4cedfbc88d5cdb7a2a53f92b34c2b16
  Preparing metadata (setup.py) ... done
  Created wheel for hypergrad: filename=hypergrad-0.1-py3-none-any.whl size=8199 sha256=2ee6ec14f29ac9104de1e10095c4465d3bc9a02975cb07284ffe5a4c8058252b
  Stored in directory: /tmp/pip-ephem-wheel-cache-6pphychk/wheels/12/18/2f/fc715468b5745149d6d37c67b27ef8a4f5de487c90d71576b5
Successfully built hypergrad


In [ ]:
!pip install pretrainedmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=2801f0a9e9a4e074dbcb81a566482253160516d9fdf19055b86fecb3b7be6060
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built pretrainedmodels


In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=1569700598872a7a08e3ce783867dc4e101873bb23b93d87bdb0556fbc9f9cdb
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
!pip install tmm

In [ ]:
#Imports
import os
import sys
import glob
import torch
import torchvision
import torchvision.models as models

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser


import pandas as pd

from hypergrad import SGDHD, AdamHD

import pretrainedmodels

from efficientnet_pytorch import EfficientNet

import ttach as tta

%matplotlib inline

In [ ]:
NAME = "SUBMISSION2"

MODEL_NAME1 = 'se_resnext101_32x4d' # could be fbresnet152 or inceptionresnetv2
MODEL_NAME2 = 'efficientnet-b0'
DIM_1 = 550
DIM_2 = 500
DIM_TEST_1 = 550
DIM_TEST_2 = 500

BATCH_SIZE = 8

NUM_EPOCHS1 = 11

random_seed = 42
shuffle_dataset = True
validation_split = .1

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_path = "/kaggle/input/ammi-2023-convnets"
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages/extraimages")

In [ ]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(DIM_1),
                                       transforms.RandomCrop(DIM_2),
                                       transforms.RandomHorizontalFlip(0.3),
                                       transforms.RandomVerticalFlip(0.3),
                                       transforms.ToTensor(),
                                       transforms.RandomErasing(0.1),
                                       transforms.Normalize(mean=mean, std=std)])

test_transforms = transforms.Compose([ transforms.Resize(DIM_TEST_1),
                                      transforms.CenterCrop(DIM_TEST_1),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, std=std)])

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, path, dim, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.dim = dim

        self.targets = []


        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.targets.append(i)

        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)

        return im.view(3, self.dim, self.dim), classCategory

class CassavaTestDataset(Dataset):
    def __init__(self, path, dim, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.indices = []
        self.dim=dim

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.indices.append(fileName.split("/")[-1])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        index = self.file_list[idx][2]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)

        return im.view(3, self.dim, self.dim), index

train_data = CassavaDataset(train_path, dim=DIM_2, transform=train_transforms)
test_data = CassavaTestDataset(test_path, dim=DIM_TEST_1, transform=test_transforms)

In [ ]:
train_data.classes

['cmd', 'cbb', 'cbsd', 'healthy', 'cgm']

In [ ]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                             sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                             sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    # Make sure the model is in evaluation mode.
    model.eval()
    # We do not need to maintain intermediate activations while testing.
    accs = []
    with torch.no_grad():

        # Loop over test data.
        for features, target in data_loader:

            # Forward pass.
            output = model(features.to(device))

            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)

            # Count number of correct predictions.
            correct = pred.cpu().eq(target.view_as(pred)).sum().item()
            total = pred.shape[0]
            accs.append(correct/total)

    # Print test accuracy.
    percent = 100. * np.mean(accs)
    st = np.std(accs)
    return percent, st

def train(model, criterion, data_loader, test_data_loader, optimizer, num_epochs, filename):
    """Simple training loop for a PyTorch model."""

    # Make sure model is in training mode.
    model.train()

    # Move model to the device (CPU or GPU).
    model.to(device)

    # Exponential moving average of the loss.
    ema_loss = None

    best_acc = 0

    print('----- Training Loop -----')
    # Loop over epochs.
    for epoch in range(num_epochs):

        # Loop over data.
        for batch_idx, (features, target) in enumerate(data_loader):

            # Forward pass.
            output = model(features.to(device))
            loss = criterion(output.to(device), target.to(device))

            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # NOTE: It is important to call .item() on the loss before summing.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01

        # Print out progress the end of epoch.
        print('----- Model Evaluation -----')
        print('Epoch: {}/{} \tTrain Loss: {:.6f}'.format(epoch+1,num_epochs, ema_loss))
        train_a, train_st = test(model,data_loader)
        test_a, test_st = test(model,test_data_loader)
        print(f'Train accuracy: ({train_a:.2f}%) with std:({train_st:.2f})')
        print(f'Test accuracy: ({test_a:.2f}%) with std:({test_st:.2f})')
        if test_a > best_acc:
            best_acc = test_a
            torch.save(model.state_dict(), filename+".pth")

    checkpoint = torch.load(filename+".pth")
    model.load_state_dict(checkpoint)
    print("------")
    test_a, test_st = test(model,test_data_loader)
    print(f'Final test accuracy: ({test_a:.2f}%) with std:({test_st:.2f})')

    return model


def generate_predictions(model,data_loader):
    model.eval()
    preds=[]
    idx=[]

    print('----- MAKING PREDICTIONS -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():

        # Loop over test data.
        for features, indices in data_loader:

            # Forward pass.
            output = model(features.to(device))

            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            for p,ind in zip(pred,indices):
                idx.append(ind)
                preds.append(p.item())

    return preds,idx

def map_to_classes(n):
    return train_data.classes[n]

In [ ]:
def get_resnext(model_name):
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    model.last_linear = nn.Linear(in_features=2048, out_features=5, bias=True)#2048

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamHD(model.parameters(), lr=1e-4, hypergrad_lr=1e-9)

    return model, criterion, optimizer

def get_eff_net(model_name, dim=1280):
    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(in_features=dim, out_features=5, bias=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamHD(model.parameters(), lr=1e-4, hypergrad_lr=1e-9)

    return model, criterion, optimizer



In [ ]:
preds = []

In [ ]:
# model1, criterion, optimizer = get_eff_net(MODEL_NAME2)
# #model1, criterion, optimizer =get_eff_net( MODEL_NAME2 , dim=1280)
# model1 = train(model1, criterion, train_loader, valid_loader, optimizer, num_epochs=NUM_EPOCHS1, filename="Resfin")

# tta_model = tta.ClassificationTTAWrapper(model1, tta.aliases.five_crop_transform(DIM_TEST_2,DIM_TEST_2))
# predictions, _ = generate_predictions(tta_model,test_loader)
# preds.append(predictions)

model1, criterion, optimizer = get_eff_net(MODEL_NAME2)
#model1, criterion, optimizer =get_eff_net( MODEL_NAME2 , dim=1280)
model1 = train(model1, criterion, train_loader, valid_loader, optimizer, num_epochs=NUM_EPOCHS1, filename="Resfin")

tta_model = tta.ClassificationTTAWrapper(model1, tta.aliases.five_crop_transform(DIM_TEST_2,DIM_TEST_2))
predictions, _ = generate_predictions(tta_model,test_loader)
preds.append(predictions)

Loaded pretrained weights for efficientnet-b0
----- Training Loop -----
----- Model Evaluation -----
Epoch: 1/11 	Train Loss: 0.570839
Train accuracy: (89.12%) with std:(0.10)
Test accuracy: (87.72%) with std:(0.11)
----- Model Evaluation -----
Epoch: 2/11 	Train Loss: 0.401023
Train accuracy: (88.73%) with std:(0.10)
Test accuracy: (87.37%) with std:(0.10)
----- Model Evaluation -----
Epoch: 3/11 	Train Loss: 0.319417
Train accuracy: (90.90%) with std:(0.09)
Test accuracy: (89.82%) with std:(0.09)
----- Model Evaluation -----
Epoch: 4/11 	Train Loss: 0.253230


In [ ]:
final_predictions = np.mean(preds,axis=0)
final_predictions.shape

(3774,)

In [ ]:
name = [test_data.file_list[i][-1].split('/')[-1] for i in range(len(test_data.file_list)) ]

In [ ]:
sample = pd.read_csv('/kaggle/input/ammi-2023-convnets/sample_submission_file.csv')
sample['Id'] = name

In [ ]:
mapping = {0: 'cmd', 1: 'cbb', 2: 'cbsd', 3: 'healthy', 4: 'cgm'}
new_preds = [mapping[int(pred)] for pred in final_predictions]
sample['Category'] = new_preds
sample.to_csv('DD10_ResNext_EfficientnetF5.csv', index=False)
sample.head()

,Category,Id
0,cgm,test-img-1448.jpg
1,cmd,test-img-768.jpg
2,cmd,test-img-3481.jpg
3,cmd,test-img-1475.jpg
4,cgm,test-img-2498.jpg
